In [30]:
import pandas as pd
import numpy as np
import os
from collections import Counter
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform

In [31]:
score_data = pd.read_csv('scores.csv', index_col=[0,1])
score_data

,,account_tenure,adview_per_month,dislike_per_month,is_churn,like_per_month,message_per_month,newfriend_per_month,post_per_month,reply_per_month,unfriend_per_month
account_id,observation_date,,,,,,,,,,
15,2020-02-09,-1.209611,0.822564,-1.051739,False,0.793760,-0.426749,-0.060075,0.796266,-1.149704,-0.543621
81,2020-02-09,-1.209611,-0.150753,-0.127564,False,-0.346823,1.419355,-0.699664,-0.418960,0.373834,-0.543621
127,2020-02-09,-1.209611,0.954385,1.117846,False,0.256470,-0.237268,-0.060075,0.135032,-0.843118,-0.543621
153,2020-02-09,-1.209611,0.605907,-0.398629,False,0.908750,-1.065580,1.474939,0.937206,-1.149704,-0.543621
177,2020-02-09,-1.209611,1.441274,1.235779,False,2.274766,-0.482624,0.579514,0.861801,0.067248,-0.543621
...,...,...,...,...,...,...,...,...,...,...,...
13132,2020-05-10,-1.243188,-1.533100,-0.780674,False,-0.517563,0.095857,-0.827582,0.057567,-0.202447,-0.543621
13141,2020-05-10,-1.243188,-0.538876,-1.433784,False,-1.749034,-0.426749,-0.699664,-0.877930,-0.319005,-0.543621
13155,2020-05-10,-1.243188,-1.867928,-1.433784,False,-1.969155,0.095857,-0.827582,-1.944690,0.511694,-0.543621


In [32]:
score_data.drop('is_churn',axis=1, inplace=True)

In [33]:
metric_columns = list(score_data.columns.values)

In [43]:
group_corr_thresh = 0.5
corr = score_data.corr()

dissimilarity = 1.0 - corr
diss_thresh = 1.0 - group_corr_thresh

#entre mas chiquito menos distance matrix redundant
dissimilarity

,account_tenure,adview_per_month,dislike_per_month,like_per_month,message_per_month,newfriend_per_month,post_per_month,reply_per_month,unfriend_per_month
account_tenure,0.000000,0.938240,0.939929,0.938234,0.944448,0.944108,0.933649,0.954141,0.972431
adview_per_month,0.938240,0.000000,0.549769,0.328146,0.865814,0.465980,0.336606,0.908610,0.997697
dislike_per_month,0.939929,0.549769,0.000000,0.544753,0.848637,0.630040,0.563513,0.882930,0.998302
like_per_month,0.938234,0.328146,0.544753,0.000000,0.889718,0.459788,0.335805,0.930220,1.003704
message_per_month,0.944448,0.865814,0.848637,0.889718,0.000000,0.919254,0.895083,0.118376,0.997971
newfriend_per_month,0.944108,0.465980,0.630040,0.459788,0.919254,0.000000,0.470477,0.956129,0.998617
post_per_month,0.933649,0.336606,0.563513,0.335805,0.895083,0.470477,0.000000,0.933506,0.998423
reply_per_month,0.954141,0.908610,0.882930,0.930220,0.118376,0.956129,0.933506,0.000000,0.997267
unfriend_per_month,0.972431,0.997697,0.998302,1.003704,0.997971,0.998617,0.998423,0.997267,0.000000


In [51]:
#1D condensed distance matrix
linkage_matrix = linkage(squareform(dissimilarity), method='single')

In [56]:
labels = fcluster(
    linkage_matrix,diss_thres,criterion='distance')

labels

array([4, 2, 3, 2, 1, 2, 2, 1, 5], dtype=int32)

In [79]:
#counts the number of elements on each cluster
cluster_count = Counter(labels)

#este diccionarion nadamas tiene los indices
cluster_order = {cluster[0] : idx for idx, cluster in enumerate(cluster_count.most_common())}

cluster_order

{np.int32(2): 0,
 np.int32(1): 1,
 np.int32(4): 2,
 np.int32(3): 3,
 np.int32(5): 4}

In [83]:
relabeled_clusters = [cluster_order[l] for l in labels]

In [101]:
relabeled_count = Counter(relabeled_clusters)
relabeled_count

Counter({0: 4, 1: 2, 2: 1, 3: 1, 4: 1})

In [104]:
relabeled_df = pd.DataFrame({
    'group' : relabeled_clusters,
    'column' : metric_columns
}).sort_values(['group','column'], ascending=[True,True])
relabeled_df

,group,column
1,0,adview_per_month
3,0,like_per_month
5,0,newfriend_per_month
6,0,post_per_month
4,1,message_per_month
7,1,reply_per_month
0,2,account_tenure
2,3,dislike_per_month
8,4,unfriend_per_month
